#1. Import modules

In [ ]:
from torchvision import transforms
from torchvision import datasets
import numpy as np
import PIL
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
from torch.utils.data.sampler import SubsetRandomSampler

from torchvision import transforms
from torchvision import datasets


### Settings

In [ ]:
import random
import utils 

batch_size = 32

# available:
# resnet18, resnet34, resnet50, se_resnet18, se_resnet34, se_resnet50, resnext
# inception_v3, se_inception_v3, xception, se_xception, inception_resnet
# densenet121, se_densenet121
model_arch = 'resnet18'

# available: 2, 4 or 5
n_classes = 5

# available devices:
# - cuda:0
# - cpu
torch_device = 'cuda:0'

# choose loss function, available:
# - cross_entropy
# - ordinal
loss_func = 'cross_entropy'

# directories
train_dataset_path = 'oai224/train'
test_dataset_path = 'oai224/test'
output_dir = 'output'

# other
epochs_num = 75

In [ ]:
def prepare_variables(model_arch, new_random_seed, classes_num, suffix='/'):
    random_seed = new_random_seed
    checkpoints_dir = output_dir + '/' + model_arch + '_' + str(random_seed) + '_' + str(classes_num) + suffix
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    
    # if you are suing GPU
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    
    torch.backends.cudnn.enabled = False 
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    
    return checkpoints_dir

#3. Split dataset to train and validation subsets

In [ ]:
from sklearn import model_selection

validation_split = .125

def prepare_dataset(train_dataset_path, test_dataset_path, frame_size):
    # split data to train and validation
    transforms_to_train = transforms.Compose([         
                  transforms.ColorJitter(brightness=.33, saturation=.33),
                  transforms.RandomHorizontalFlip(p=0.5),
                  transforms.RandomAffine(degrees=(-10, 10), scale=(0.9, 1.10)),
                  transforms.Resize(frame_size), 

                  transforms.ToTensor(),
                  transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                ])

    train_dataset = datasets.ImageFolder(train_dataset_path, transform=transforms_to_train)
    targets = train_dataset.targets

    train_idx, valid_idx = model_selection.train_test_split(
        np.arange(len(train_dataset.targets)), test_size=validation_split, random_state=42, shuffle=True, stratify=targets)
    
    print('dataset\n---')
    print(np.unique(np.array(train_dataset.targets)[train_idx], return_counts=True))
    print(np.unique(np.array(train_dataset.targets)[valid_idx], return_counts=True))

    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(valid_idx)
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler, drop_last=True)
    val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=val_sampler, drop_last=True)

    return train_loader, val_loader


# 4. Prepare other parts.

In [ ]:
from torch.utils.data.sampler import Sampler

class SubsetSampler(Sampler):
    r"""Samples elements with given indices sequentially

    Arguments:
        indices (ndarray): indices of the samples to take
    """

    def __init__(self, indices):
        self.indices = indices

    def __iter__(self):
        return (self.indices[i] for i in range(len(self.indices)))

    def __len__(self):
        return len(self.indices)
    
    
def evaluate_model(model, dataset, indices):
    """
    Computes predictions and ground truth labels for the indices of the dataset
    
    Returns: 
    predictions: np array of ints - model predictions
    grount_truth: np array of ints - actual labels of the dataset
    """
    model.eval() # Evaluation mode
    
    predictions = []
    ground_truth = []
    
    part_size = 64 # len(indices)
    subset_sampler = SubsetSampler(indices)
    val_loader = torch.utils.data.DataLoader(dataset, batch_size=part_size, sampler=subset_sampler)
    
    for i_step, (x, y) in enumerate(val_loader):
        x_gpu = x.to(torch_device)
        y_gpu = y.to(torch_device)
        
        probs = model(x_gpu)  

        pred = torch.argmax(probs, 1)  
        gt = y

        predictions.extend(pred.cpu().tolist())
        ground_truth.extend(gt.cpu().tolist())

    return predictions, ground_truth

In [ ]:
# prepare dataset to check test accuracy
def check_accuracy_on_test_set(model, path, device):
    transforms_to_test = transforms.Compose([
                  transforms.Resize(frame_size), 
                  transforms.ToTensor(),
                  transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                ])

    test_dataset = datasets.ImageFolder(test_dataset_path, transform=transforms_to_test)
    test_idx = list(range(len(test_dataset.imgs)))
    test_sampler = SubsetRandomSampler(test_idx)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, sampler=test_sampler)

    with torch.no_grad():
        test_accuracy, _, _, _ = utils.compute_accuracy(model, test_loader, device)
        print("test accuracy:", test_accuracy)
        append_to_file(path + 'test_accuracy.txt', float(test_accuracy))
    
    return test_dataset, test_idx


In [ ]:
import matplotlib.pyplot as plt

def save_train_graph(path):
    train_history = read_file(path + 'train_history.txt')
    val_history = read_file(path + 'val_history.txt')

    # визуализация
    plt.plot(train_history)
    plt.plot(val_history)
    plt.savefig(path + 'graph.png')
    

In [ ]:
import sklearn.metrics as metrics
import matplotlib.pyplot as plt

from tabulate import tabulate
from sklearn.metrics import precision_recall_fscore_support as score

def performance_matrix(true,pred):
    precision = metrics.precision_score(true,pred,average='weighted')
    recall = metrics.recall_score(true,pred,average='weighted') # average='weighted'
    accuracy = metrics.accuracy_score(true,pred)
    f1_score = metrics.f1_score(true,pred,average='weighted')
    print('Confusion Matrix:\n', metrics.confusion_matrix(true, pred))
    print('\nMean Precision: {} Recall: {}, Accuracy: {}, f1_score: {}'.format(precision*100,recall*100,accuracy*100,f1_score*100))

    with open(checkpoints_dir + 'score_mean.txt', 'w') as f:
        f.write('Mean Precision: {} Recall: {}, Accuracy: {}, f1_score: {}\n'.format(precision*100,recall*100,accuracy*100,f1_score*100))
        f.write('\nConfusion Matrix:\n')
        f.write(str(metrics.confusion_matrix(true, pred)))


In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'
    # Compute confusion matrix
    cm = metrics.confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    #classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')
    
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    
    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    fig.savefig(checkpoints_dir + 'conf_mat2.png')
    return ax


In [ ]:
def weighted_loss5(outputs, labels):
    softmax_op = torch.nn.Softmax(1)
    prob_pred = softmax_op(outputs)

    # prepare weights
    init_weights = np.array([[1, 3, 5, 7, 9],
                            [3, 1, 3, 5, 7],
                            [5, 3, 1, 3, 5],
                            [7, 5, 3, 1, 3],
                            [9, 7, 5, 3, 1]], dtype=float)

    cls_weights = init_weights + 1.0
    np.fill_diagonal(cls_weights, 0)

    # calc
    batch_num, class_num = outputs.size()
    class_hot = np.zeros([batch_num, class_num], dtype=np.float32)
    labels_np = labels.data.cpu().numpy()
    for ind in range(batch_num):
        class_hot[ind, :] = cls_weights[labels_np[ind], :]
    class_hot = torch.from_numpy(class_hot)
    class_hot = torch.autograd.Variable(class_hot).cuda()

    loss = torch.sum((prob_pred * class_hot)**2) / batch_num
    # loss = torch.mean(prob_pred * class_hot)

    return loss

def weighted_loss4(outputs, labels):
    softmax_op = torch.nn.Softmax(1)
    prob_pred = softmax_op(outputs)

    # prepare weights
    init_weights = np.array([[1, 3, 5, 7],
                            [3, 1, 3, 5],
                            [5, 3, 1, 3],
                            [7, 5, 3, 1]], dtype=float)
        
    cls_weights = init_weights + 1.0
    np.fill_diagonal(cls_weights, 0)

    # calc
    batch_num, class_num = outputs.size()
    class_hot = np.zeros([batch_num, class_num], dtype=np.float32)
    labels_np = labels.data.cpu().numpy()
    for ind in range(batch_num):
        class_hot[ind, :] = cls_weights[labels_np[ind], :]
    class_hot = torch.from_numpy(class_hot)
    class_hot = torch.autograd.Variable(class_hot).cuda()

    loss = torch.sum((prob_pred * class_hot)**2) / batch_num
    # loss = torch.mean(prob_pred * class_hot)

    return loss

In [ ]:
import re
import os
import utils

def append_to_file(filename, val):
    with open(filename, 'a') as f:
        f.write("%s\n" % val)

def read_file(filename):
    lines = []
    with open(filename, 'r') as f:
        lines = [float(line.strip()) for line in f]

    return lines

def train_model(model, train_loader, val_loader, loss, optimizer, num_epochs, lr_scheduler, output_dir): 
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    start_epoch = 0

    open(output_dir + 'loss_history.txt', 'w').close()
    open(output_dir + 'train_history.txt', 'w').close()
    open(output_dir + 'val_history.txt', 'w').close()
    open(output_dir + 'best_accuracy.txt', 'w').close()
    print("start traning from scratch...")

    best_val_accuracy = 0

    checkpoints = []
    loss_history = []
    train_history = []
    val_history = []
    for epoch in range(num_epochs):
        model.train() # Enter train mode
        
        loss_accum = 0
        correct_samples = 0
        total_samples = 0

        # process batches
        for i_step, (x, y) in enumerate(train_loader): 
            x_gpu = x.to(torch_device)
            y_gpu = y.to(torch_device)
            prediction = model(x_gpu)    

            loss_value = loss(prediction, y_gpu)

            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            
            _, indices = torch.max(prediction, 1)
            correct_samples += torch.sum(indices == y_gpu)
            total_samples += y.shape[0]
            
            loss_accum += loss_value

        # check accuracy
        ave_loss = loss_accum / i_step
        train_accuracy = float(correct_samples) / total_samples

        val_accuracy = 0.0
        with torch.no_grad():
          val_accuracy, _, _, _ = utils.compute_accuracy(model, val_loader, torch_device)

        # write marks to files
        append_to_file(output_dir + 'loss_history.txt', float(ave_loss))
        append_to_file(output_dir + 'train_history.txt', train_accuracy)
        append_to_file(output_dir + 'val_history.txt', val_accuracy)

        # update learning rate
        if lr_scheduler is not None:
          lr_scheduler.step()
        
        stage = epoch + start_epoch

        if val_accuracy > best_val_accuracy:
          best_val_accuracy = val_accuracy
          
          # next better model
          model_save_name2 = 'model.ckpt-' + str(stage)
          torch.save(model.state_dict(), output_dir + F"{model_save_name2}")
          checkpoints.append(model_save_name2)
          if len(checkpoints) > 3:
            os.remove(output_dir + F"{checkpoints[0]}")
            checkpoints.pop(0)
          
          # best model
          model_save_name = 'best_model.ckpt'
          torch.save(model.state_dict(), output_dir + F"{model_save_name}")

          append_to_file(output_dir + 'best_accuracy.txt', best_val_accuracy)
          print("update best model with val. accuracy %f on stage %d" % (best_val_accuracy, stage))

        print("epoch %d; average loss: %f, train accuracy: %f, val accuracy: %f" % (stage, ave_loss, train_accuracy, val_accuracy))
        
    print("final best accuracy: %f" % (best_val_accuracy))

    return loss_history, train_history, val_history
        

In [ ]:
# prepare loss
if loss_func == "ordinal":
    loss = weighted_loss5 if n_classes == 5 else weighted_loss4
else: # cross_entropy
    loss = nn.CrossEntropyLoss()
    if torch.cuda.is_available() and torch_device == 'cuda:0':
        loss.type(torch.cuda.FloatTensor)
    else:
        loss.type(torch.FloatTensor)

# train model
for seed in [21, 42, 84]:
    print(' ')
    print('start training for seed ' + str(seed))
    
    # prepare model
    cnn_model, frame_size, classes_header = utils.load_model(model_arch, n_classes);
 
    # prepare model to cpu/gpu calc.
    if torch.cuda.is_available() and torch_device == 'cuda:0':
        cnn_model.type(torch.cuda.FloatTensor)
    else:
        cnn_model.type(torch.FloatTensor)
    
    cnn_model = cnn_model.to(torch_device)

    # prepare checkpoint
    checkpoints_dir = prepare_variables(model_arch, seed, len(classes_header), suffix='/') # '_weighted/'
    print('output directory: ' + checkpoints_dir)

    print(' ')
    train_loader, val_loader = prepare_dataset(train_dataset_path, test_dataset_path, frame_size);
      
    optimizer = optim.Adam(cnn_model.parameters(), lr=1e-3, weight_decay=1e-4)
    # optimizer = optim.SGD(cnn_model.parameters(), lr=1e-4, momentum=0.9, nesterov=True) 
    # optimizer = optim.Adam(cnn_model.parameters(), lr=1e-4, weight_decay=1e-4) # default: weight_decay=1e-4

    # decrease learning rate by 5% every 5 epochs
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.95) 

    print(' ')
    loss_history, train_history, val_history = train_model(
        cnn_model, 
        train_loader, 
        val_loader, 
        loss, 
        optimizer, 
        epochs_num, 
        lr_scheduler,
        output_dir = checkpoints_dir)

    # постобработка
    save_train_graph(checkpoints_dir)
    
    print('')
    state_dict = torch.load(checkpoints_dir + 'best_model.ckpt')
    cnn_model.load_state_dict(state_dict) 
    
    test_dataset, test_idx = check_accuracy_on_test_set(cnn_model, checkpoints_dir, torch_device)
    
    # evaluate model on validation
    gt = []
    predictions = []
    with torch.no_grad():
        predictions, gt = evaluate_model(cnn_model, test_dataset, test_idx)
    
    print(' ')
    performance_matrix(gt, predictions)

    precision, recall, fscore, support = score(gt, predictions)

    print(' ')
    print('rows is precision, recall, fscore and support:')
    print(tabulate([precision, recall, fscore, support], headers=classes_header, tablefmt='orgtbl'))

    with open(checkpoints_dir + 'score.txt', 'w') as f:
        f.write('rows is precision, recall, fscore and support:\n')
        f.write(tabulate([precision, recall, fscore, support], headers=classes_header, tablefmt='orgtbl'))
      
    print(' ')
    plot_confusion_matrix(gt, predictions, classes=classes_header,title='Confusion matrix')  
        
    print('\n==\n')
    